# Applying Classification Modeling
The goal of this week's assessment is to find the model which best predicts whether a person will default on their loan. In doing so, we want to utilize all of the different tools we have learned over the course: data cleaning, EDA, feature engineering/transformation, feature selection, hyperparameter tuning, and model evaluation. 

Dataset: The dataset comes customers default payments in Taiwan. More information about the dataset and columns are found in the link below.

https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients#


You will fit three different models (KNN, Logistic Regression, and Decision Tree Classifier) and use gridsearch to find the best hyperparameters for those models. Then you will compare the performance of those three models on a test set to find the best one.  


## Process/Expectations

#### You will be working in pairs for this assessment; please have ONE notebook and be prepared to explain how you worked in your pair.
1. Clean up your data set so that you can do EDA. This includes handling null values, categorical variables, removing unimportant columns, and removing outliers.
2. Perform EDA to identify opportunities to create new features.
    - [Great Example of EDA for classification](https://www.kaggle.com/stephaniestallworth/titanic-eda-classification-end-to-end) 
    - [Using Pairplots with Classification](https://towardsdatascience.com/visualizing-data-with-pair-plots-in-python-f228cf529166)
3. Create polynomial and/or interaction features. You must also create at least 2 new features that are not interactions or polynomial transformations. For example, you can create a new dummy variable that based on the value of a continuous variable (billamount6 >2000) or take the average of some past amounts.
4. Perform some feature selction. This can happen beforehand using F-scores, or you can do it as part of your model building process by looking at the weights of your regularized logistic regression or feature importance of your decision tree.  
5. You must fit each of the three models to your data and tune at least 1 hyperparameter per model. 
6. After identifying the best hyperparameters for each model, fit those models to the test set and identify the best model overall using the evaluation metric of your choice.
7. Present your best model.

In [18]:
# import libraries

import pandas as pd

import seaborn as sns

## 1. Data Cleaning

In [3]:
df = pd.read_csv('student_data.csv')


In [4]:
df['default payment next month'].value_counts()

0    18691
1     5308
Name: default payment next month, dtype: int64

In [5]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,2873,350000,1,1,2,37,-2,-2,-2,-2,...,466,466,316,316,316,466,466,316,316,0
1,3598,50000,2,2,1,37,2,2,2,0,...,13026,13268,13497,5500,0,580,600,600,600,0
2,27623,50000,2,1,2,23,-1,-1,-1,-1,...,4800,9810,660,2548,2321,4800,9810,660,2980,0
3,6874,20000,1,3,1,56,0,0,0,0,...,13784,13420,13686,1508,1216,1116,0,490,658,0
4,6444,110000,2,2,2,32,0,0,0,0,...,108829,110557,106082,5400,5400,4100,4100,4100,4200,0


In [6]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')

In [12]:
df.EDUCATION.value_counts()

2    11188
1     8442
3     3994
5      223
4      100
6       41
0       11
Name: EDUCATION, dtype: int64

In [27]:
df.groupby('default payment next month')['MARRIAGE'].value_counts()

default payment next month  MARRIAGE
0                           2           10058
                            1            8399
                            3             194
                            0              40
1                           2            2677
                            1            2559
                            3              69
                            0               3
Name: MARRIAGE, dtype: int64

## 2. EDA

In [10]:
df.columns



Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month', 'pay_ahead'],
      dtype='object')

## 3. Feature Engineering

In [29]:
#feature engineer a new column that tells us how many payments a person pays ahead 



df['pay_ahead'] = df.PAY_0+df.PAY_2+df.PAY_3+df.PAY_4+df.PAY_5+df.PAY_6

In [30]:
#create dummy variables
df['debt_status'] = df['BILL_AMT1'] - df['BILL_AMT6']

In [31]:
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,pay_ahead,debt_status
0,2873,350000,1,1,2,37,-2,-2,-2,-2,...,316,316,316,466,466,316,316,0,-12,0
1,3598,50000,2,2,1,37,2,2,2,0,...,13497,5500,0,580,600,600,600,0,6,27497
2,27623,50000,2,1,2,23,-1,-1,-1,-1,...,660,2548,2321,4800,9810,660,2980,0,-6,2770
3,6874,20000,1,3,1,56,0,0,0,0,...,13686,1508,1216,1116,0,490,658,0,2,-2839
4,6444,110000,2,2,2,32,0,0,0,0,...,106082,5400,5400,4100,4100,4100,4200,0,0,2077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23994,7886,50000,2,1,2,24,0,0,0,-2,...,0,7522,0,0,0,0,0,1,-6,12806
23995,7750,250000,1,3,1,44,-1,-1,-1,-1,...,576,3924,4482,43676,38166,576,2389,1,-6,6250
23996,14379,50000,1,3,2,26,2,0,0,2,...,48721,1800,4000,0,3500,2000,0,1,8,-6894
23997,23664,110000,2,2,1,38,0,0,0,0,...,-1288,5000,10000,5048,2000,39958,52000,1,0,196725


In [36]:
df1 = pd.get_dummies(data=df, columns=['SEX', 'EDUCATION', 'MARRIAGE'], prefix=['sex', 'edu', 'marr'])

In [38]:
df1.columns

Index(['ID', 'LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5',
       'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
       'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default payment next month',
       'pay_ahead', 'debt_status', 'sex_1', 'sex_2', 'edu_0', 'edu_1', 'edu_2',
       'edu_3', 'edu_4', 'edu_5', 'edu_6', 'marr_0', 'marr_1', 'marr_2',
       'marr_3'],
      dtype='object')

## 4. Feature Selection

## 5. Model Fitting and Hyperparameter Tuning
KNN, Logistic Regression, Decision Tree

## 6. Model Evaluation

## 7. Final Model